In [92]:
from dotenv import dotenv_values
from openai import OpenAI
import time

In [93]:
OPENAI_API_KEY = dotenv_values(".env")["OPENAI_API_KEY"]
client = OpenAI(api_key=OPENAI_API_KEY)

In [94]:
GPT_MODEL = "gpt-3.5-turbo-1106"

# Overview
input: prompt of what the website should be.

output: code + ai execution of the website. 
1. Create webpage
   1. Executive detailing necessary components
   2. For each component: Provide previously solved context (summary of what's been done)
      1. Assistant asking questions
      2. Executive answering questions
      3. REPEAT for some steps
   3. Combine components
   4. Host
2. Test webpage
   1. Test connectivity using agent + code interpreter
   2. Test UI locally, and with semantic interpretation of the code compared to initial prompt


# Notes
Context window is likely large enough to keep everything in there without summary. d

In [189]:
class Developer:
    def __init__(self, client, assistant_id):
        self.client = client
        self.assistant_id = assistant_id

    def ask_question(self, context):
        thread = self.client.beta.threads.create()

        message = self.client.beta.threads.messages.create(
            thread_id=thread.id,
            role="user",
            content=f"Context: {context}\n As the developer, if necessary to implement, ask a clarifying question about the current component. Otherwise, respond with No Question.",
        )

        run = self.client.beta.threads.runs.create(
            thread_id=thread.id,
            assistant_id=self.assistant_id,
            instructions="Ask a specific question to clarify details about the current component",
        )

        while True:
            time.sleep(1)
            run_status = self.client.beta.threads.runs.retrieve(
                thread_id=thread.id, run_id=run.id
            )
            if run_status.status == "completed":
                break
            elif run_status.status == "failed":
                raise Exception("Run failed")

        messages = self.client.beta.threads.messages.list(thread_id=thread.id)
        question = messages.data[0].content[0].text.value

        return question

    def implement_component(self, context):
        thread = self.client.beta.threads.create()

        message = self.client.beta.threads.messages.create(
            thread_id=thread.id,
            role="user",
            content=f"Based on this context, provide code for ONLY the current component: {context}",
        )

        run = self.client.beta.threads.runs.create(
            thread_id=thread.id,
            assistant_id=self.assistant_id,
            instructions="Provide the code to implement only the current component based on the provided context and specifications. NEVER provide code unprompted",
        )

        while True:
            time.sleep(1)
            run_status = self.client.beta.threads.runs.retrieve(
                thread_id=thread.id, run_id=run.id
            )
            if run_status.status == "completed":
                break
            elif run_status.status == "failed":
                raise Exception("Run failed")

        messages = self.client.beta.threads.messages.list(thread_id=thread.id)
        implementation = messages.data[0].content[0].text.value

        return implementation

    def integrate_components(self, implementations, webpage_idea):
        """Integrates all component codes into a single coherent implementation"""
        thread = self.client.beta.threads.create()
        message = self.client.beta.threads.messages.create(
            thread_id=thread.id,
            role="user",
            content=f"""\nTask: Combine all these implementations into a single, coherent final application to {webpage_idea} Ensure that:
            
            1. All necessary imports are at the top of the file
            2. Code is organized logically
            3. There are no duplicate or conflicting code
            4. Resolve conflicting or redundant pieces of code. 
            """,
        )

        run = self.client.beta.threads.runs.create(
            thread_id=thread.id,
            assistant_id=self.assistant_id,
            instructions="Integrate all the provided implementations into a single, coherent application. Resolve any conflicts and ensure the final code is complete and ready to run. ",
        )

        while True:
            time.sleep(1)
            run_status = self.client.beta.threads.runs.retrieve(
                thread_id=thread.id, run_id=run.id
            )
            if run_status.status == "completed":
                break
            elif run_status.status == "failed":
                raise Exception("Run failed")

        messages = self.client.beta.threads.messages.list(thread_id=thread.id)
        integrated_implementation = messages.data[0].content[0].text.value

        return integrated_implementation


class Executive:
    def __init__(self, client, assistant_id):
        self.client = client
        self.assistant_id = assistant_id

    def answer_question(self, context, question):
        thread = self.client.beta.threads.create()

        message = self.client.beta.threads.messages.create(
            thread_id=thread.id,
            role="user",
            content=f"Context: {context} Developer's Question: {question}\n As the senior advisor, answer with specificity the developer's question about this component. If there is no question, then do not respond. Do not provide unprompted information.",
        )

        run = self.client.beta.threads.runs.create(
            thread_id=thread.id,
            assistant_id=self.assistant_id,
            instructions="Answer the developer's question with specificity",
        )

        while True:
            time.sleep(1)
            run_status = self.client.beta.threads.runs.retrieve(
                thread_id=thread.id, run_id=run.id
            )
            if run_status.status == "completed":
                break
            elif run_status.status == "failed":
                raise Exception("Run failed")

        messages = self.client.beta.threads.messages.list(thread_id=thread.id)
        answer = messages.data[0].content[0].text.value

        return answer

    def generate_summary(self, previous_components, implementation):
        thread = self.client.beta.threads.create()

        message = self.client.beta.threads.messages.create(
            thread_id=thread.id,
            role="user",
            content="Provide an explicit summary of what has been implemented. Include only and all implemented components, and provide variable names and implementation details using natural language.",
        )

        run = self.client.beta.threads.runs.create(
            thread_id=thread.id,
            assistant_id=self.assistant_id,
            instructions=f"Summarize what has been implemented in the {previous_components}, and in the current component {implementation}"
        )

        while True:
            time.sleep(1)
            run_status = self.client.beta.threads.runs.retrieve(
                thread_id=thread.id, run_id=run.id
            )
            if run_status.status == "completed":
                break
            elif run_status.status == "failed":
                raise Exception("Run failed")

        messages = self.client.beta.threads.messages.list(thread_id=thread.id)
        summary = messages.data[0].content[0].text.value

        return summary

In [190]:
def communicative_dehallucination(executive, developer, summary, component, max_iter=1):
    """
    Implements a communicative dehallucination process for software development.

    Args:
        executive (object): The executive assistant object for answering questions.
        developer (object): The developer assistant object for asking questions and implementing.
        summary (str): A summary of previously implemented components.
        component (str): The current component to be implemented.
        max_iter (int, optional): Maximum number of Q&A iterations.

    Returns:
        tuple: A tuple containing:
            - implementation (str): The generated implementation of the component.
            - context (str): The full context including all Q&A pairs.
            - summary (str): A concise summary of what has been achieved for this component.
    """
    
    context = f"""Previous Components summarized: {summary}\nCurrent Component: {component}"""
    print(f'Context: \n{context}\n')
    
    # Iterative Q&A process
    for i in range(max_iter):
        # Developer asks a question
        question = developer.ask_question(context)
        print(f"Developer's question: {question}\n")
        
        # Executive answers the question
        answer = executive.answer_question(context, question)
        print(f"Executive's answer: {answer}\n")
        
        # Update context with new Q&A pair
        context += f"\n {i+1}: Clarifying Question about current component: {question}\n Clarifying Answer about current component: {answer}\n"
    
    # Developer implements component based on clarified context
    implementation = developer.implement_component(context)
    print(f"Implementation:\n{implementation}\n")
    
    # Generate a summary of what has been achieved
    summary = executive.generate_summary(summary, implementation)
    print(f"Summary: {summary}")
    
    return implementation, context, summary

In [191]:
executive = client.beta.assistants.create(
    name="Executive",
    instructions="You are a senior software developer. You break down tasks and provide high-level instructions using natural language.",
    model=GPT_MODEL,
)

developer = client.beta.assistants.create(
    name="Developer",
    instructions="You are a web developer. You create and deploy Flask webpages based on instructions, asking questions when necessary",
    model=GPT_MODEL,
)

developer_assistant = Developer(client, developer.id)
executive_assistant = Executive(client, executive.id)

In [192]:
WEBPAGE_IDEA = "Provide the code to quickstart a basic builtin Flask server. The Flask server should only show Hello World"
thread = client.beta.threads.create()

initial_prompt = client.beta.threads.messages.create(
    thread_id=thread.id, role="user", content=WEBPAGE_IDEA
)

In [193]:
run = client.beta.threads.runs.create_and_poll(
    thread_id=thread.id,
    assistant_id=executive.id,
    # List, in natural language, only the essential code components needed to fulfill the user's request.
    instructions="""
    List, in natural language, only the essential code components needed to fulfill the user's request.
   
    Your response must:
    1. Include only core components.
    2. Put each new component on a new line (not numbered).
    3. Focus on the conceptual steps of specific code elements or function calls
    4. Be comprehensive, covering all necessary components
    5. Use technical terms appropriate for the specific programming language and framework.
    6. Naturally sequence components, so that later components are dependent on previous ones. 
    
    Important:
    - Assume all dependencies are already installed but not imported.
    - Do not include dependency installations.
    - Focus solely on the code components needed to implement the functionality.
    - NEVER provide code example
    - ALWAYS ensure all necessary components are present
   
    Example format:
    1. [Natural language specification of the specific code component or function call]
    2. [Natural language specification of the specific code component or function call]
    ...
    """,
)


In [194]:
if run.status == "completed":
    messages = client.beta.threads.messages.list(thread_id=thread.id)
    for message in messages:
        print(message.content[0].text.value, '\n\n')
else:
    print(run.status)
    
# Parse the components from the executive's response
messages = client.beta.threads.messages.list(thread_id=thread.id)
    
message = client.beta.threads.messages.retrieve(
    thread_id=thread.id, message_id=messages.data[0].id
)
components = message.content[0].text.value.split('\n')
components = [comp.strip() for comp in components if comp.strip()]

# Initialize previous_components
summary = ""
all_implementations = []


1. Import the Flask class from the flask module.
2. Create an instance of the Flask class.
3. Define a route using the route decorator, specifying the homepage using '/'.
4. Create a function to return "Hello World".
5. Run the application using the run method, specifying the host and port if needed. 


Provide the code to quickstart a basic builtin Flask server. The Flask server should only show Hello World 




In [195]:
for i, component in enumerate(components):
    print(f"\nProcessing Component {i+1}: \n{component}")
    
    # Use communicative_dehallucination for each component
    implementation, context, summary = communicative_dehallucination(
        executive_assistant, 
        developer_assistant, 
        summary, 
        component, 
        max_iter=1 
    )
    
    # Add the implementation to our list
    all_implementations.append(implementation)


Processing Component 1: 
1. Import the Flask class from the flask module.
Context: 
Previous Components summarized: 
Current Component: 1. Import the Flask class from the flask module.

Developer's question: No Question

Executive's answer: Since the developer did not provide a question about importing the Flask class from the flask module, there is no specific answer to provide in this case.

Implementation:
```python
from flask import Flask
```

Summary: In the provided code, a Flask application has been implemented using the "Flask" module. A Flask instance named "app" has been created using the statement "app = Flask(__name__)". This instance is used to define the routes and views for the web application.

Processing Component 2: 
2. Create an instance of the Flask class.
Context: 
Previous Components summarized: In the provided code, a Flask application has been implemented using the "Flask" module. A Flask instance named "app" has been created using the statement "app = Flask(__

In [196]:
final_code = developer_assistant.integrate_components(all_implementations, WEBPAGE_IDEA)

In [197]:
print(final_code)

Here's the finalized application by combining all the provided implementations into a coherent Flask server:

```python
from flask import Flask

app = Flask(__name__)

@app.route('/')
def hello_world():
    return 'Hello, World!'

if __name__ == '__main__':
    app.run()
```

Make sure to save this code in a file like `app.py` and then run it with `python app.py` in your terminal. This will start the Flask server, and you can access the "Hello, World!" message by navigating to `http://127.0.0.1:5000/` in your web browser.


# Testing 
Try to run the code with an agent using code interpreter, and then try and ping it maybe?